In [171]:
# Import necessary libraries
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import tkinter as tk
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import END
from transformers import pipeline

In [172]:
# Function for extractive text summarization
def extractive_summarization(text, num_sentences=3):
    # Load spaCy model
    nlp = spacy.load('en_core_web_sm')
    
    # Tokenization and removing stopwords
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc 
              if not token.is_stop and not token.is_punct and token.text != '\n']

    # Calculating word frequency
    word_freq = Counter(tokens)
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = word_freq[word] / max_freq
    
    # Sentence tokenization
    sent_token = [sent.text for sent in doc.sents]

    # Calculating sentence score based on word frequency
    sent_score = {}
    for sent in sent_token:
        for word in sent.split():
            if word.lower() in word_freq.keys():
                if sent not in sent_score.keys():
                    sent_score[sent] = word_freq[word]
                else:
                    sent_score[sent] += word_freq[word]

    # Select top-scoring sentences
    summarized_sentences = nlargest(num_sentences, sent_score, key=sent_score.get)

    return " ".join(summarized_sentences)

In [173]:
# Function for abstractive text summarization
def abstractive_summarization(text):
    summarizer = pipeline("summarization", model='t5-base', tokenizer='t5-base', framework='pt')
    summary = summarizer(text, max_length=100, min_length=10, do_sample=False)
    return summary[0]['summary_text']


In [174]:
# Function to summarize text using extractive approach
def summarize_extractive():
    # Get text from the text box
    text = text_box.get("1.0", "end-1c")
    
    # Summarize text using extractive approach
    summarized_text = extractive_summarization(text, num_sentences=int(num_sentences_entry.get()))

    # Display summarized text in the result box
    result_box.delete(1.0, END)
    result_box.insert(END, summarized_text)

In [175]:
# Function to summarize text using abstractive approach
def summarize_abstractive():
    # Get text from the input text box
    text = text_entry.get("1.0", "end-1c")

    # Summarize the text using abstractive approach
    summary = abstractive_summarization(text)

    # Update the output text box with the summary
    output_text.delete("1.0", "end")
    output_text.insert("1.0", summary)

In [176]:
# GUI setup for extractive summarization
root = tk.Tk()
root.title("Text Summarizer (Extractive)")

# Text box for input
text_box = scrolledtext.ScrolledText(root, width=50, height=10, wrap=tk.WORD)
text_box.pack(pady=10)

# Entry field for the number of sentences
num_sentences_label = tk.Label(root, text="Number of Sentences:")
num_sentences_label.pack()
num_sentences_entry = tk.Entry(root, width=10)
num_sentences_entry.insert(END, "3")  # Default value
num_sentences_entry.pack()

# Button to summarize using extractive approach
summarize_button = tk.Button(root, text="Summarize (Extractive)", command=summarize_extractive)
summarize_button.pack(pady=5)

# Result box for output
result_box = scrolledtext.ScrolledText(root, width=50, height=5, wrap=tk.WORD)
result_box.pack(pady=10)

# Run the Tkinter event loop for extractive summarization
root.mainloop()

In [177]:
# GUI setup for abstractive summarization
window = tk.Tk()
window.title("Text Summarizer (Abstractive)")

# Create input text box
text_entry = tk.Text(window, height=10, width=60)
text_entry.pack(pady=10)

# Entry field for the number of sentences
num_sentences_label = tk.Label(window, text="Number of Sentences:")
num_sentences_label.pack()
num_sentences_entry = tk.Entry(window, width=10)
num_sentences_entry.insert(END, "3")  # Default value
num_sentences_entry.pack()

# Button to summarize using abstractive approach
summarize_button = tk.Button(window, text="Summarize (Abstractive)", command=summarize_abstractive)
summarize_button.pack()

# Create output text box
output_text = tk.Text(window, height=10, width=60)
output_text.pack(pady=10)

# Run the Tkinter event loop for abstractive summarization
window.mainloop()

In [178]:
# Original text
text = """In a world often dominated by negativity, it's important to remember the power of kindness and compassion. Small acts of kindness have the ability to brighten someone's day, uplift spirits, and create a ripple effect of positivity that can spread far and wide. Whether it's a smile to a stranger, a helping hand to a friend in need, or a thoughtful gesture to a colleague, every act of kindness has the potential to make a difference in someone's life.Beyond individual actions, there is also immense power in collective efforts to create positive change. When communities come together to support one another, incredible things can happen. From grassroots initiatives to global movements, people are uniting to tackle pressing social and environmental issues, driving meaningful progress and inspiring hope for a better future.It's also important to recognize the strength that lies within each and every one of us. We all have the ability to make a positive impact, no matter how small our actions may seem. By tapping into our innate compassion and empathy, we can cultivate a culture of kindness and empathy that enriches our lives and those around us.So let's embrace the power of kindness, and strive to make the world a better place one small act at a time. Together, we can create a brighter, more compassionate future for all."""

# Perform extractive summarization
extractive_summary = extractive_summarization(text)

# Perform abstractive summarization
abstractive_summary = abstractive_summarization(text)

# Display results
print("Extractive Summary:")
print(extractive_summary)
print("\nAbstractive Summary:")
print(abstractive_summary)

Extractive Summary:
Small acts of kindness have the ability to brighten someone's day, uplift spirits, and create a ripple effect of positivity that can spread far and wide. From grassroots initiatives to global movements, people are uniting to tackle pressing social and environmental issues, driving meaningful progress and inspiring hope for a better future. Whether it's a smile to a stranger, a helping hand to a friend in need, or a thoughtful gesture to a colleague, every act of kindness has the potential to make a difference in someone's life.

Abstractive Summary:
small acts of kindness can brighten someone's day, uplift spirits, and create a ripple effect of positivity . when communities come together to support one another, incredible things can happen . we all have the ability to make a positive impact, no matter how small our actions may seem .


In [179]:
import nltk
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import time

# Load NLTK resources
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [180]:

# Example text and reference summary
input_text = """In a world often dominated by negativity, it's important to remember the power of kindness and compassion. Small acts of kindness have the ability to brighten someone's day, uplift spirits, and create a ripple effect of positivity that can spread far and wide. Whether it's a smile to a stranger, a helping hand to a friend in need, or a thoughtful gesture to a colleague, every act of kindness has the potential to make a difference in someone's life. Beyond individual actions, there is also immense power in collective efforts to create positive change. When communities come together to support one another, incredible things can happen. From grassroots initiatives to global movements, people are uniting to tackle pressing social and environmental issues, driving meaningful progress and inspiring hope for a better future. It's also important to recognize the strength that lies within each and every one of us. We all have the ability to make a positive impact, no matter how small our actions may seem. By tapping into our innate compassion and empathy, we can cultivate a culture of kindness and empathy that enriches our lives and those around us. So let's embrace the power of kindness, and strive to make the world a better place one small act at a time. Together, we can create a brighter, more compassionate future for all."""
reference_summary = "Small acts of kindness have the ability to brighten someone's day, uplift spirits, and create a ripple effect of positivity that can spread far and wide. From grassroots initiatives to global movements, people are uniting to tackle pressing social and environmental issues, driving meaningful progress and inspiring hope for a better future. Whether its a smile to a stranger, a helping hand to a friend in need, or a thoughtful gesture to a colleague, every act of kindness has the potential to make a difference in someones life."

In [181]:
# Extractive Summarization Function
def extractive_summarization(text, num_sentences=3):
    sentences = nltk.sent_tokenize(text)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    sentence_scores = similarity_matrix.sum(axis=1)
    top_sentences_indices = sentence_scores.argsort()[-num_sentences:][::-1]
    summary = [sentences[i] for i in top_sentences_indices]
    return ' '.join(summary)

# Abstractive Summarization Function
def abstractive_summarization(text):
    summarizer = pipeline("summarization", model='t5-base', tokenizer='t5-base', framework='pt')
    summary = summarizer(text, max_length=100, min_length=10, do_sample=False)
    return summary[0]['summary_text']


# New function for abstractive summarization using output of extractive summarization
def abstractive_summarization_from_extractive(extractive_summary):
    return abstractive_summarization(extractive_summary)

In [182]:

# Function to evaluate ROUGE scores
def evaluate_rouge(summary, reference):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    return scores[0]  # Return the first score (ROUGE-1)

# Function to compare efficiency
def compare_efficiency(text):
    start_time = time.time()
    extractive_summary = extractive_summarization(text)
    extractive_time = time.time() - start_time
    
    start_time = time.time()
    abstractive_summary = abstractive_summarization(text)
    abstractive_time = time.time() - start_time
    
    

    
   
    
    return extractive_time, abstractive_time




In [183]:
# Perform extractive summarization
extractive_summary = extractive_summarization(input_text)

# Perform abstractive summarization
abstractive_summary = abstractive_summarization(input_text)

# Perform abstractive summarization using output of extractive summarization
abstractive_summary_new = abstractive_summarization_from_extractive(extractive_summary)

# Evaluate ROUGE scores
extractive_rouge_scores = evaluate_rouge(extractive_summary, reference_summary)
abstractive_rouge_scores = evaluate_rouge(abstractive_summary, reference_summary)
abstractive_rouge_new_scores = evaluate_rouge(abstractive_summary_new, reference_summary)


# Compare efficiency
extractive_time, abstractive_time = compare_efficiency(input_text)

# Print results
print("Extractive Summary:")
print(extractive_summary)
print("\nAbstractive Summary:")
print(abstractive_summary)
print("\nAbstractive new Summary:")
print(abstractive_summary_new)
print("\nExtractive ROUGE Scores:")
print(extractive_rouge_scores)
print("\nAbstractive ROUGE Scores:")
print(abstractive_rouge_scores)
print("\nAbstractive new ROUGE Scores:")
print(abstractive_rouge_new_scores)
print("\nExtractive Summarization Time:", extractive_time)
print("Abstractive Summarization Time:", abstractive_time)


Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)


Extractive Summary:
In a world often dominated by negativity, it's important to remember the power of kindness and compassion. So let's embrace the power of kindness, and strive to make the world a better place one small act at a time. Small acts of kindness have the ability to brighten someone's day, uplift spirits, and create a ripple effect of positivity that can spread far and wide.

Abstractive Summary:
small acts of kindness can brighten someone's day, uplift spirits, and create a ripple effect of positivity . when communities come together to support one another, incredible things can happen . it's important to recognize the strength that lies within each and every one of us .

Abstractive new Summary:
small acts of kindness can brighten someone's day, uplift spirits, and create a ripple effect of positivity . sally kohn: we can make the world a better place one small act at a time .

Extractive ROUGE Scores:
{'rouge-1': {'r': 0.4090909090909091, 'p': 0.5625, 'f': 0.473684205650